<H1 align="center"><span style="color:firebrick;font-size:200%;padding-left:2%"><B><i>Recommendation System</i></B></span><H1>
    <HR style="width:50%;background-color:gray;padding:2px;" align="center"></HR>
    <H2 align="center"><span style="color:indigo">Book:</span></H2>
    <H3 align="center"><span style="color:green">Machine Learning using Python</span></H3>
    <H2 align="center"><span style="color:indigo">Author:</span></H2>
    <H3 align="center"><span style="color:green">Manaranjan Pradhan</span></H3>
    <H3 align="center"><span style="color:green">U. Dinesh Kumar</span></H3>
    <H2 align="center"><span style="color:indigo">Publisher:</span></H2>
    <H3 align="center"><span style="color:green">Wiley India Pvt. Ltd</span></H3>
    <H2 align="center"><span style="color:indigo">Presented By:</span></H2>
    <H3 align="center"><span style="color:green">Shubham Jain</span></H3>
    
![Recommendation System](https://images.xenonstack.com/usecase/xenonstack-deep-learning-based-recommendation-system.png)

# What is Recommendation System?
- Set of algorithms which recommend most relevant items to users based on their preferences
- Acts on behavioral data, such as customer's previous purchase, ratings or reviews, to predict the likelihood of buying a new product or service.

#### Three algorithms used widely for building recommendation systems:
1. Association Rules
2. Collaborative Filtering
3. Matrix Factorization

# Association Rules Mining
- Finds combinations of items that frequently occur together in orders or baskets.
- Items that frequently occur together are called itemsets.
- Itemsets help to discover relations between items that people buy together
- Application of Association Rules: Market Basket Analysis (MBA)
- MBA: Technique used by retailers to find relations between items purchased by customers

## Problem with Association Rules Mining: Computational Cost
As number of unqiue items sold by seller increases, number of associations can increase exponentially.
### Solution to above problem: Apriori Algorithm
- Proposed by Agrawal and Srikant
- Rules are represented as: {Antecedent} --> {Consequent}. Eg: {Bread} --> {Butter}
- Calculates the probability of an item being present in a frequent itemset, given that another item or items is present ([Recommendation System, Oracle](https://docs.oracle.com/cd/B28359_01/datamine.111/b28129/algo_apriori.htm#i1007565))
- The Apriori principle tells us that subsets of frequent itemsets are frequent. Thus, if we find an infrequent itemset, which we'll call {X}, then it must be the case that {X, Y} is also infrequent, so we may eliminate it without computing its support.

## Metrics to Measure Association Rules
### 1. Support:
- Indicates how frequently the items in a given association rule occur together.
- Apriori algorithm computes support for each item independently and eliminates items with support less than minimum support, say, 0.01.
- <u>Formula</u>:
$$Support(X, Y) = \frac{N_{XY}}{N}$$
$N_{XY}$: Number of baskets in which items $X$ and $Y$ appear together<br>
$N$: Total Number of Baskets<br>
$N_{X}$: Number of baskets in which $X$ appears<br>
$N_{Y}$: Number of baskets in which $Y$ appears<br>

## 2. Confidence
- Indicates the probability of both the antecedent and the consequent appearing in the same transaction
- <u>Formula</u>:
$$Confidence(X-->Y) = P(Y|X) = \frac{N_{XY}}{N_{X}}$$
$P(Y|X)$: Conditional probability of $Y$ given $X$<br>
$X$: Antecedent item<br>
$Y$: Consequent item

## 3. Lift
- Degree of association between two items
- Indicates the strength of a rule over the random co-occurrence of the antecedent and the consequent, given their individual support
- Provides information about the improvement, the increase in probability of the consequent given the antecedent
- <u>Lift Values</u>:<br>
1) Lift = 1: Items are independent<br>
2) Lift < 1: Items are substitutes of each other (Purchasing one product decreases probability of purchasing other product)<br>
3) Lift > 1: Purchase of one product increases probability of purchasing other product. It is a necessary condition of generating association rules.<br>

- Ranges from 0 to infinity

- <u>Formula</u>:
$$Lift = \frac{Support(X, Y)}{Support(X) x Support(Y)} = \frac{N_{XY}}{N_{X}N_{Y}}$$

## 4. Leverage
- Leverage is similar to lift, but it is easy to interpret
- Ranges between -1 to 1
- <u>Formula</u>:
$$Leverage(X --> Y) = Support(X, Y) - Support(X) * Support(Y)$$ 

## 5. Conviction
- More Complicated and Less Intuitive than Leverage
- <u>Formula</u>:
$$Conviction(X --> Y) = \frac{Support(X)*Support(Y^\complement)}{Support(X, Y^\complement)}$$

# Applying Association Rules
- Dataset: groceries.csv, taken from DEPARTMENT OF STATISTICS AND BIOSTATISTICS, CAL STATE EASY BAY
- Each line in the dataset is an order and contains a variable number of items

In [59]:
# Import all required libraries
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules 
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation 
from surprise import Dataset, Reader, KNNBasic, accuracy 
from surprise.model_selection import cross_validate 
from surprise.model_selection.search import GridSearchCV 
from surprise import SVD

## Loading the Dataset

In [2]:
all_txns = []
#open the file
with open('groceries.csv') as f:
    #read each line
    content = f.readlines()
    #Remove white space from the beginning and end of the line
    txns = [x.strip() for x in content]
    # Iterate through each line and create a list of transactions
    for each_txn in txns:
        #Each transaction will contain a list of item in the transaction
        all_txns.append( each_txn.split(',') )

all_txns[0:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

### Encoding Transactions

The transactions need to be converted in a tabular or matrix form whose dimensions are $M$ X $N$, where $M$ = Total Number of Transactions and $N$ = All Unique Items across all Transactions. For this, we will use TransactionEncoder from <i>mlxtend</i> library. It will one-hot encode all transactions where each transactions will consist of values 'True' or 'False' as shown below.

#### Reference:
1. http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
2. https://campus.datacamp.com/courses/market-basket-analysis-in-python/introduction-to-market-basket-analysis-1?ex=9

In [3]:
# Instantiate transaction encoder and identify unique items
encoder = TransactionEncoder().fit(all_txns)

# One-hot encode transactions
onehot = encoder.transform(all_txns)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
onehot.head(12)

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [4]:
print("Dimensions:\n\nNumber of Transactions =", onehot.shape[0], "\nNumber of Items =", onehot.shape[1])

Dimensions:

Number of Transactions = 9835 
Number of Items = 169


### Generating Association Rules

Now, we will prune all itemsets that have support value less than 0.02 as shown below:

In [5]:
print("Resulting Frequent Itemsets (Min. Support = 0.02)")
frequent_itemsets = apriori(onehot,
                            min_support=0.02,
                            use_colnames=True) 
frequent_itemsets.sample(10, random_state=42)

Resulting Frequent Itemsets (Min. Support = 0.02)


,support,itemsets
18,0.053279,(curd)
45,0.037824,(salty snack)
47,0.098526,(shopping bags)
89,0.035892,"(other vegetables, tropical fruit)"
4,0.080529,(bottled beer)
40,0.088968,(pastry)
62,0.024199,"(rolls/buns, bottled water)"
107,0.025826,"(yogurt, root vegetables)"
31,0.037417,(long life bakery product)
55,0.071683,(whipped/sour cream)


Now, the above itemsets will be sent to association_rules to generate rules based on lift with minimum threshold = 1.

In [6]:
print("Resulting Association Rules Based on Lift (Min. Threshold = 1)")
rules = association_rules(frequent_itemsets, # itemsets
                          metric="lift", # lift
                          min_threshold=1) 
rules.sample(10)

Resulting Association Rules Based on Lift (Min. Threshold = 1)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23,(citrus fruit),(yogurt),0.082766,0.139502,0.021657,0.261671,1.875752,0.010111,1.165467
4,(rolls/buns),(bottled water),0.183935,0.110524,0.024199,0.131564,1.190373,0.003870,1.024228
27,(other vegetables),(domestic eggs),0.193493,0.063447,0.022267,0.115081,1.813824,0.009991,1.058350
42,(pastry),(other vegetables),0.088968,0.193493,0.022572,0.253714,1.311235,0.005358,1.080695
44,(pip fruit),(other vegetables),0.075648,0.193493,0.026131,0.345430,1.785237,0.011494,1.232118
68,(pastry),(whole milk),0.088968,0.255516,0.033249,0.373714,1.462587,0.010516,1.188729
9,(bottled water),(whole milk),0.110524,0.255516,0.034367,0.310948,1.216940,0.006126,1.080446
123,(yogurt),"(whole milk, other vegetables)",0.139502,0.074835,0.022267,0.159621,2.132979,0.011828,1.100890
99,(soda),(shopping bags),0.174377,0.098526,0.024606,0.141108,1.432194,0.007425,1.049578
7,(bottled water),(soda),0.110524,0.174377,0.028978,0.262190,1.503577,0.009705,1.119017


Lets sort these association rules based on confidence in descending order.

In [7]:
rules.sort_values('confidence', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
121,"(yogurt, other vegetables)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340
17,(butter),(whole milk),0.055414,0.255516,0.027555,0.497248,1.946053,0.013395,1.480817
24,(curd),(whole milk),0.053279,0.255516,0.026131,0.490458,1.919481,0.012517,1.461085
116,"(root vegetables, other vegetables)",(whole milk),0.047382,0.255516,0.023183,0.489270,1.914833,0.011076,1.457687
114,"(whole milk, root vegetables)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
28,(domestic eggs),(whole milk),0.063447,0.255516,0.029995,0.472756,1.850203,0.013783,1.412030
109,(whipped/sour cream),(whole milk),0.071683,0.255516,0.032232,0.449645,1.759754,0.013916,1.352735
91,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401
50,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
33,(frozen vegetables),(whole milk),0.048094,0.255516,0.020437,0.424947,1.663094,0.008149,1.294636


From above output, we can infer that the probability that a customer buys 'whole milk' given he/she has bought both 'yogurt' and 'other vegetables' is 0.51. 

## Advantages of Association Rule Mining
1. Transaction data, which is used for generating rules, is always available and mostly clean.
2. The rules generated are simple and can be interpreted.

## Disadvantage of Association Rule Mining
- Does not take into account the preference or rating given by a customer for an item

## Applications of Association Rule Mining
1. Product Recommendations
2. Fraud Detection from Transaction Sequence
3. Medical Diagnosis
4. Weather Prediction

# Collaborative Filtering
- Based in the notion of similarity
- If two users A & B have purchased the same product and have rated them similarly on a common rating scale, then, A and B can be considered similar in their buying and preference behavior. Hence, if A buys a new product and rates it highly, then, that product can be recommended to B.
- Measures of similarity: Jaccard Coefficient, Cosine Similarity, Euclidean Distance and Pearson Correlation.

![CF](https://d4datascience.files.wordpress.com/2016/07/cf.png)

## 1. Cosine Similarity
- Measures the similarity between two vectors of an inner product space
- Measured by the cosine of the angle between two vectors
- Determines whether two vectors are pointing in roughly the same direction
- Often used to measure document similarity in text analysis

![CS](https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d)

## 2. Jaccard Coefficient
- Compares members for two sets to see which members are shared and which are distinct
- A measure of similarity for the two sets of data, with a range from 0% to 100%
- The higher the percentage, the more similar the two populations.

![JC](https://i.ytimg.com/vi/Ah_4xqvS1WU/maxresdefault.jpg)

## 3. Euclidean Distance
- "Ordinary" straight-line distance between two points in Euclidean space

![ED](https://miro.medium.com/proxy/1*IVy5Ozk6e3sQtG_wuKo19A.jpeg)

## 4. Pearson Correlation
- Measures linear correlation between two variables X and Y
- Has a value between +1 and −1

![PC](https://www.statisticshowto.com/wp-content/uploads/2012/10/pearson-2-small.png)

## Variation in Collaborative Filtering
1. User - Based Similarity: Find K similar users based on common items they have bought.
2. Item - Based Similarity: Find K similar items based on common users who have bought those items.

![CFV](https://d4datascience.files.wordpress.com/2016/07/recomfde.jpg?w=748)

## User - Based Similarity
[MovieLens](https://grouplens.org/datasets/movielens/) from GroupLens is used for finding similar users based on common movies the users have watched and how they have rated those movies. The ratings are on the scale of 1 to 5. The dataset has the following features:
1. userId
2. MovieId
3. Rating
4. timestamp

We will remove timestamp as it has no use in our analysis.

### Loading the Dataset

In [8]:
rating_df = pd.read_csv("ml-latest-small/ml-latest-small/ratings.csv")[['userId', 'movieId', 'rating']]
rating_df.head(7)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0


In [9]:
print("Number of Unique Users:", rating_df.userId.nunique(), "\nNumber of Unique Movies:", rating_df.movieId.nunique())

Number of Unique Users: 610 
Number of Unique Movies: 9724


Now, we will pivot above dataframe to form a matrix whose columns are movies and rows are users i.e. it will have dimensions of 610 X 9724.

In [10]:
user_movies_df = rating_df.pivot(index='userId', columns='movieId', values='rating').reset_index(drop=True)
user_movies_df.index = rating_df.userId.unique()
user_movies_df.fillna(0, inplace = True)
user_movies_df.head(7)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculating Cosine Similarity between Users
- pairwise_distance() from sklearn is used to compute distance between all pairs of users.
- Cosine Similarity closer to 1 means users are very similar and closer to 0 means users are very dissimilar.

In [11]:
user_sim = 1 - pairwise_distances( user_movies_df.values, metric="cosine" )
#Store the results in a dataframe
user_sim_df = pd.DataFrame( user_sim )
# set the index and column names to user ids (0 to 610)
user_sim_df.index = rating_df.userId.unique()
user_sim_df.columns = rating_df.userId.unique() 
user_sim_df.head(7)

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792
6,0.128152,0.025333,0.003936,0.088491,0.300349,1.000000,0.075843,0.370488,0.013904,0.020385,...,0.021415,0.448927,0.098002,0.396582,0.104541,0.102123,0.162182,0.178809,0.214234,0.052668
7,0.158744,0.027585,0.000000,0.115120,0.108342,0.075843,1.000000,0.114885,0.099463,0.132099,...,0.206405,0.125182,0.103664,0.062025,0.219586,0.200035,0.186114,0.323541,0.090840,0.193219


Here, similarity between user 1 and user 1 is 1 which is obvious. However, we want similarity between distincy users. So, we will set diagonal values to 0 as shown below.

In [12]:
np.fill_diagonal(user_sim, 0)
user_sim_df.head(7)

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,0.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,...,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,0.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,...,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,0.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,...,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,0.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,...,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,0.000000,0.300349,0.108342,0.429075,0.000000,0.030611,...,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792
6,0.128152,0.025333,0.003936,0.088491,0.300349,0.000000,0.075843,0.370488,0.013904,0.020385,...,0.021415,0.448927,0.098002,0.396582,0.104541,0.102123,0.162182,0.178809,0.214234,0.052668
7,0.158744,0.027585,0.000000,0.115120,0.108342,0.075843,0.000000,0.114885,0.099463,0.132099,...,0.206405,0.125182,0.103664,0.062025,0.219586,0.200035,0.186114,0.323541,0.090840,0.193219


### Filtering Similar Users

We will find similar users based on maximum value of each column as shown below:

In [13]:
user_sim_df.idxmax(axis=1).head(7)

1    266
2    366
3    313
4    391
5    470
6    117
7    239
dtype: int64

So,  user 1 is similar to user 266, user 2 is similar to user 366 and so on. Let's confirm by seeing the actual similarity values for User 1 and User 2.

In [14]:
user_sim_df.iloc[0:2, [264 ,265, 266, 267, 365, 366, 367, 368, 369]] 

,265,266,267,268,366,367,368,369,370
1,0.193046,0.357408,0.163540,0.153292,0.067588,0.151562,0.345127,0.173543,0.212594
2,0.012474,0.000000,0.028248,0.015622,0.300074,0.031699,0.008637,0.016431,0.034816


User 1 indeed has high similarity with User 266 and this is true for other pair of users too. To validate this, we will see which movies they have seen in common and rated very similarly.

### Loading Movies Dataset

In [15]:
movies_df = pd.read_csv( "ml-latest-small/ml-latest-small/movies.csv" ).drop('genres', axis = 1)
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### Finding Common Movies of Similar Users

Here, we will fetch movies that user 1 and user 266 have watched and rated atleast 4

In [16]:
def get_user_similar_movies( user1, user2 ):
    # Inner join between movies watched between two users will give the common movies watched.
    common_movies = rating_df[rating_df.userId == user1].merge(
    rating_df[rating_df.userId == user2],
             on = "movieId",
             how = "inner" )
    # join the above result set with movies details
    return common_movies.merge( movies_df, on = 'movieId' ) 

In [17]:
common_movies = get_user_similar_movies( 1, 265 ) 
common_movies[(common_movies.rating_x >= 4.0) &
((common_movies.rating_y >= 4.0))] 

,userId_x,movieId,rating_x,userId_y,rating_y,title
1,1,260,5.0,265,5.0,Star Wars: Episode IV - A New Hope (1977)
3,1,480,4.0,265,4.0,Jurassic Park (1993)
6,1,593,4.0,265,5.0,"Silence of the Lambs, The (1991)"
7,1,608,5.0,265,4.0,Fargo (1996)
9,1,923,5.0,265,4.0,Citizen Kane (1941)
11,1,1196,5.0,265,5.0,Star Wars: Episode V - The Empire Strikes Back...
12,1,1198,5.0,265,5.0,Raiders of the Lost Ark (Indiana Jones and the...
14,1,1240,5.0,265,5.0,"Terminator, The (1984)"
15,1,1270,5.0,265,5.0,Back to the Future (1985)
20,1,1954,5.0,265,5.0,Rocky (1976)


There are 18 movies in total that user 1 and user 266 have watched and rated highly. Similarly, let's see for user 2 and user 261 which are very dissimilar.

In [18]:
common_movies = get_user_similar_movies( 2, 261 ) 
common_movies[(common_movies.rating_x >= 4.0) &
((common_movies.rating_y >= 4.0))] 

,userId_x,movieId,rating_x,userId_y,rating_y,title
2,2,58559,4.5,261,5.0,"Dark Knight, The (2008)"
3,2,79132,4.0,261,4.5,Inception (2010)


Indeed, User 2 and User 261 are dissimilar as they have seen only two movies that they have rated higher.

## Challenges to User-Based Similarity: Cold Start
- Does not work for new users
- Have to wait until the new user buys a few items and rates them
- Solution: <B>Item Based Similarity</B>

## Item Based Similarity
If two movies A and B have been watched by several users and rated very similarly, then both A & B can be similar in taste. In other words, if a user watches movie A, then he or she is very likely to watch movie B and vice versa

### Calculating Cosine Similarity between Movies

Here, we will pivot the rating_df dataframe in the form of a matrix where users are columns and movies are rows. Each cell $a_{ij}$ is filled with rating given by user $j$ to movie $i$.

In [19]:
rating_mat = rating_df.pivot( index='movieId', columns='userId', values = "rating" ).reset_index(drop=True)
# fill all NaNs with 0
rating_mat.fillna( 0, inplace = True )
# Find the correlation between movies
movie_sim = 1 - pairwise_distances( rating_mat.values, metric="correlation" )
# Fill the diagonal with 0, as it repreresent the auto-correlation of movies
movie_sim_df = pd.DataFrame( movie_sim ) 
movie_sim_df.head(7)

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,1.000000,0.231327,0.173213,-0.028917,0.192474,0.192686,0.143743,0.085477,0.177245,0.183382,...,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906,-0.028906
1,0.231327,1.000000,0.191945,0.071269,0.200526,0.158341,0.127569,0.141540,-0.021045,0.285086,...,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291,-0.018291
2,0.173213,0.191945,1.000000,0.067143,0.370171,0.196442,0.351513,0.296897,0.275812,0.136916,...,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729,-0.011729
3,-0.028917,0.071269,0.067143,1.000000,0.167910,0.053755,0.258075,0.148726,-0.016025,0.056000,...,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138,-0.004138
4,0.192474,0.200526,0.370171,0.167910,1.000000,0.215503,0.429890,0.265777,0.308085,0.110833,...,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456,-0.011456
5,0.192686,0.158341,0.196442,0.053755,0.215503,1.000000,0.148109,0.114707,0.167909,0.251343,...,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712,-0.017712
6,0.143743,0.127569,0.351513,0.258075,0.429890,0.148109,1.000000,0.255120,0.124458,0.129010,...,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033,-0.012033


### Finding Most Similar Movies

In [20]:
def get_similar_movies( movieid, topN = 5 ):
    movieidx = movies_df[movies_df.movieId == movieid].index[0]
    movies_df['similarity'] = movie_sim_df.iloc[movieidx]
    top_n = movies_df.sort_values( ["similarity"], ascending = False )[0:topN]
    return top_n 

Let's find movies similar to 'Godfather' (movieId = 858).

In [27]:
get_similar_movies(858) 

,movieId,title,similarity
659,858,"Godfather, The (1972)",1.000000
921,1220,"Blues Brothers, The (1980)",0.769390
913,1212,"Third Man, The (1949)",0.560246
895,1192,Paris Is Burning (1990),0.496048
827,1088,Dirty Dancing (1987),0.442128


So, if someone sees 'Godfather', other movies can be recommended to him based on similarity in movies.

# Using Surprise Library
- Provides API to build recoomendation system
- Various ready - to - use prediction algorithms such as neighborhood methods and matrix factorization.
- Tools to evaluate, analyze and compare the performance of the algorithms

In [34]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rating_df[['userId', 'movieId', 'rating']], reader=reader) 

## User Based SimilarityAlgorithm
### KNN Basic
1. K: Max. number of neighbors to take into account for aggregation
2. min_k: Min. number of neighbors to take into account for aggregation
3. sim_options = dictionary of similarity measure ('name') and user_based (True for User based similarity; otherwise False)

The following code finds similar movies based on pearson correlation:

In [32]:
item_based_cosine_sim = {'name': 'pearson',
                         'user_based': True}
knn = KNNBasic(k= 20,
               min_k = 5,
               sim_options = item_based_cosine_sim) 

### 5-Fold Cross Validation

In [36]:
cv_results = cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=False) 
print("\n\n------------------------------\nMean Accuracy:", 
      np.mean( cv_results.get('test_rmse') ), "  Std. of Accuracies:", np.std( cv_results.get('test_rmse') ))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


------------------------------
Mean Accuracy: 0.9716787061830955   Std. of Accuracies: 0.0024745837996589374


### Finding the best model through GridSearchCV

In [46]:
param_grid = {'k': [10, 20],
              'sim_options': {'name': ['cosine', 'pearson'],
              'user_based': [True, False]}
             }
grid_cv = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, refit=True)
grid_cv.fit(data)
# best RMSE score
print("\n\nBest RMSE Score:", grid_cv.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print("\nParameters corresponding to best RMSE:\n", grid_cv.best_params['rmse']) 

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

The best model is user-based collaborative filtering with cosine similarity and 20 similar users. Detailed results of grid search are printed below:

In [47]:
results_df = pd.DataFrame.from_dict(grid_cv.cv_results)
results_df[['param_k', 'param_sim_options', 'mean_test_rmse', 'rank_test_rmse']]

,param_k,param_sim_options,mean_test_rmse,rank_test_rmse
0,10,"{'name': 'cosine', 'user_based': True}",0.985993,4
1,10,"{'name': 'cosine', 'user_based': False}",1.025680,8
2,10,"{'name': 'pearson', 'user_based': True}",0.985760,3
3,10,"{'name': 'pearson', 'user_based': False}",1.012653,7
4,20,"{'name': 'cosine', 'user_based': True}",0.973354,1
5,20,"{'name': 'cosine', 'user_based': False}",0.994952,6
6,20,"{'name': 'pearson', 'user_based': True}",0.974460,2
7,20,"{'name': 'pearson', 'user_based': False}",0.986720,5


### Making Predictions

$r_{i}$ = rating of nearest user $u_{i}$
<br>$K$ = Number of Neighbors
<br>$s_{i}$ = Similarity between the user and the neighbor

The prospective rating given by a user for a movie is given by:

$$r=\frac{\sum_{i = 1}^{k}r_{i}s_{i}}{\sum_{i = 1}^{k}r_{i}}$$

Below we have predicted rating of user 1 for MovieId 2:

In [55]:
print("Rating of User 1 for Movie 2:", grid_cv.predict( 1, 2 ).est) 

Rating of User 1 for Movie 2: 3.7233653693643416


# Matrix Factorization
- There are latent factors that determine why a user rates a movie, and the way he/she rates
- Latent factors can be anything: actors, story, genre etc
- Difficult to determine what these latent factors represent
- A matrix with size (n, m), where n = no. of users and m = no. of movies, can be factorized into (n, k) and (k, m) matrices, where k = no. of latent factors.
- Singular vector Decomposition (SVD): One of the popular techniques for matrix factorization

In [57]:
# Use 10 factors for building the model
svd = SVD( n_factors = 10 )
cv_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8596  0.8752  0.8669  0.8792  0.8701  0.8702  0.0068  
Fit time          2.66    2.18    2.74    2.21    2.44    2.45    0.23    
Test time         0.42    0.41    0.18    0.19    0.17    0.28    0.11    
